### Library Imports 

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import pymc.sampling_jax
import matplotlib.pyplot as plt
import arviz as az
import pickle
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc

### Defining functions

In [ ]:
def plot_mean_roc(y_true, posterior_samples, title='Mean ROC curve with 95% CI'):
    fpr_grid = np.linspace(0, 1, 64)
    tpr_interpolated = []
    aucs = []
    
    
    for i in range(posterior_samples.shape[1]):
        y_scores = posterior_samples[i]
        fpr, tpr, _ = roc_curve(y, y_scores)
        tpr_interpolated.append(np.interp(fpr_grid, fpr, tpr))
        aucs.append(auc(fpr, tpr))
    
    # Convertir en array numpy pour faciliter les calculs
    tpr_interpolated = np.array(tpr_interpolated)
    
    # Calculer la TPR moyenne et son intervalle de confiance à 95%
    mean_tpr = tpr_interpolated.mean(axis=0)
    std_tpr = tpr_interpolated.std(axis=0)
    ci_lower = np.percentile(tpr_interpolated, 2.5, axis=0)
    ci_upper = np.percentile(tpr_interpolated, 97.5, axis=0)
    
    # Calculer la moyenne des AUC
    mean_auc = np.mean(aucs)
    lower_auc = np.percentile(aucs, 2.5)
    upper_auc = np.percentile(aucs, 97.5)

    # Create the title with the mean AUC and 95% CI
    title = f'Mean ROC curve with 95% CI (AUC = {mean_auc:.2f}, 95% CI: {lower_auc:.2f} - {upper_auc:.2f})'

    
    # Tracer la courbe ROC moyenne avec les intervalles de confiance
    plt.plot(fpr_grid, mean_tpr, color='blue', label='Mean ROC')
    plt.fill_between(fpr_grid, ci_lower, ci_upper, color='blue', alpha=0.2, label='95% CI')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
def calculate_mean_f1(y_true, posterior_samples, threshold=0.5):
    """
    Calcule et affiche la moyenne des F1 scores à partir des échantillons postérieurs.
    
    :param y_true: np.array, vecteur de labels binaires de taille (n_samples,)
    :param posterior_samples: np.array, matrice de taille (n_samples, n_estimations)
    :param threshold: float, seuil pour convertir les scores en labels binaires
    :return: float, moyenne des F1 scores
    """
    f1_scores = []

    for i in range(posterior_samples.shape[1]):
        y_scores = posterior_samples[i]
        y_pred = (y_scores >= threshold).astype(int)
        f1_scores.append(f1_score(y_true, y_pred))

    mean_f1 = np.mean(f1_scores)
    print(f'Mean F1 Score: {mean_f1:.2f}')
    return mean_f1

In [ ]:
def calculate_mean_confusion_matrix(y_true, posterior_samples, threshold=0.5): 
    """
    Calcule et affiche la moyenne des matrices de confusion à partir des échantillons postérieurs.
    
    :param y_true: np.array, vecteur de labels binaires de taille (n_samples,) 
    :param posterior_samples: np.array, matrice de taille (n_samples, n_estimations)
    :param threshold: float, seuil pour convertir les scores en labels binaires
    :return: np.array, matrice de confusion moyenne 
    """

    
    confusion_matrices = [] 

    for i in range(posterior_samples.shape[1]): 
        y_scores = posterior_samples[i] 
        y_pred = (y_scores >= threshold).astype(int)
        cm = confusion_matrix(y_true, y_pred)
        confusion_matrices.append(cm) 

    mean_confusion_matrix = np.mean(confusion_matrices, axis=0)
    print(f'Mean Confusion Matrix:\n{mean_confusion_matrix}')
    return mean_confusion_matrix

In [ ]:
def calculate_information_criteria(trace, model, y):
    """
    Calculate AIC, BIC, and WAIC for a PyMC model using arviz.InferenceData.
    
    Parameters:
    trace (arviz.InferenceData): Posterior samples from the model.
    model (pymc.Model): The PyMC model object.
    y (array-like): Observed data.
    
    Returns:
    dict: A dictionary containing AIC, BIC, and WAIC values.
    """
    
    # Check if log_likelihood already exists in trace
    if 'log_likelihood' not in trace.groups():
        # Compute log-likelihood if not already computed
        pm.compute_log_likelihood(trace, model=model)
    
    # Extract log-likelihood
    log_likelihood = trace.log_likelihood['y_obs'].values
    
    # Check for and remove any infinite values
    finite_log_likelihood = log_likelihood[np.isfinite(log_likelihood)]
    
    if len(finite_log_likelihood) == 0:
        logging.error("All log-likelihood values are non-finite.")
        return {'AIC': np.nan, 'BIC': np.nan, 'WAIC': np.nan}
    
    if len(finite_log_likelihood) < len(log_likelihood):
        logging.warning(f"Removed {len(log_likelihood) - len(finite_log_likelihood)} non-finite log-likelihood values.")
    
    parameter_names = [var for var in trace.posterior.data_vars if not var.endswith('_') and var != 'p_i']
    
    # Number of parameters
    k = len(parameter_names)
    
    # Number of data points
    n = len(y)
    
    # Compute mean log-likelihood
    mean_log_likelihood = np.mean(finite_log_likelihood)
    
    # Compute AIC
    aic = 2 * k - 2 * mean_log_likelihood
    
    # Compute BIC
    bic = k * np.log(n) - 2 * mean_log_likelihood
    
    # Compute WAIC
    try:
        waic = az.waic(trace)
        waic_value = waic
    except Exception as e:
        logging.error(f"Error computing WAIC: {str(e)}")
        waic_value = np.nan
        
    return {
        'AIC': aic,
        'BIC': bic,
        'WAIC': waic_value
    }

### Dataset

In [ ]:
# Read in data and display first 5 lines
data = pd.read_csv("/mnt/d/Last_attempt/proximity_data.csv")
data.head()

In [ ]:
y = data["infected"].to_numpy()
t0 = data['time_between_0.05_and_0.5m'].to_numpy()
t1 = data['time_between_0.5_and_1m'].to_numpy()
t2 = data['time_between_1_and_2m'].to_numpy()
t3 = data['time_above_2m'].to_numpy()

### Models

1. model 1:

$$p_i = 1 - (1-p)^{t_0} \cdot (1-\alpha_{1} p)^{t_1} \cdot (1-\alpha_{2} p)^{t_2} \cdot (1-\alpha_{3} p)^{t_3} \qquad where \qquad \alpha_3 < \alpha_2 < \alpha_1, \quad \alpha_1 < 1$$

2. model 2:

$$p_i = 1 - (1-p)^{t_0} \cdot (1-\alpha_{1} p)^{t_1}  \qquad where \qquad  \alpha_1 < 1$$

3. model 3:

$$p_i = 1 - (1- p)^{t_2} \cdot (1-\alpha_{1} p)^{t_3} \qquad where \qquad  \alpha_1 < 1$$

4. model 4:

$$p_i = 1 - (1- p)^{t_1} \cdot (1-\alpha_{1} p)^{t_2} \cdot (1-\alpha_{2} p)^{t_3} \qquad where \qquad  \alpha_2 < \alpha_1 < 1  $$

5. model 5:
$$p_i = 1 - (1-p)^{t_0} \cdot (1-\alpha_{1} p)^{t_1} \cdot (1-\alpha_{2} p)^{t_2}  \qquad where \qquad  \alpha_2 < \alpha_1 < 1$$



### Sampling

In [ ]:
with pm.Model() as model1:
    # Priors
    p = pm.Beta('p', 1, 1)

    alpha_1 = pm.Uniform('alpha_1', 0, 1)
    alpha_2 = pm.Uniform('alpha_2', 0, 1)
    alpha_3 = pm.Uniform('alpha_3', 0, 1)


    p_i = 1 - (1 - p)**t0 * (1 - alpha_1 * p)**t1 * (1 - alpha_1* alpha_2 * p)**t2 * (1 - alpha_1 * alpha_2* alpha_3 * p)**t3


    alpha_12 = pm.Deterministic('alpha_12', alpha_1* alpha_2)

    alpha_123 = pm.Deterministic('alpha_123',alpha_1 * alpha_2* alpha_3)

    
    p_i = pm.Deterministic('p_i', p_i)

    # Ind = pm.Deterministic('Ind', int((alpha_1 > alpha_2) and (alpha_2 > alpha_3)))

    # Likelihood
    y_true = pm.Bernoulli('y_obs', p=p_i, observed=y)


    init = {'p': 0.001}

with model1:
    trace1 = pm.sampling.jax.sample_blackjax_nuts(draws=10000, tune=1000, chains=10, target_accept=0.96, initvals=init)

In [ ]:
%%time
with pm.Model() as model2:
    # Priors
    p = pm.Beta('p', 1, 1)
    alpha_1 = pm.Uniform('alpha_1', 0, 1)    
    
    p_i = 1 - (1 - p)**t0 * (1 - alpha_1 * p)**t1 
    p_i = pm.Deterministic('p_i', p_i)

    # Likelihood
    y_true = pm.Bernoulli('y_obs', p=p_i, observed=y)

    init = {'p': 0.001}

with model2:
    trace2 = pm.sampling.jax.sample_blackjax_nuts(draws=10000, tune=1000, chains=10, target_accept=0.95, initvals=init)

In [ ]:
with pm.Model() as model3:
    # Priors
    p = pm.Beta('p', 1, 1)
    # alpha_2 = pm.Uniform('alpha_2', 0, 1)
    alpha_3 = pm.Uniform('alpha_3', 0, 1)    
    p_i = 1 - (1 - p)**t2 * (1 - alpha_3*p)**t3
    p_i = pm.Deterministic('p_i', p_i)

    # Likelihood
    y_true = pm.Bernoulli('y_obs', p=p_i, observed=y)

    init = {'p': 0.001}

with model3:
    trace3 = pm.sampling.jax.sample_blackjax_nuts(draws=10000, tune=1000, chains=10, target_accept=0.95, initvals=init)

In [ ]:
%%time
with pm.Model() as model4:
    # Priors

    p = pm.Beta('p', 1, 1)
    alpha_2 = pm.Uniform('alpha_2', 0, 1) 
    alpha_3 = pm.Uniform('alpha_3', 0, 1) 

    p_i = 1 - (1 - p)**t1 * (1 - alpha_2 * p)**t2 * (1 - alpha_3 * p)**t3

    
    p_i = pm.Deterministic('p_i', p_i)

    # Likelihood
    y_true = pm.Bernoulli('y_obs', p=p_i, observed=y)

    init = {'p': 0.001}

with model4:
    trace4 = pm.sampling.jax.sample_blackjax_nuts(draws=10000, tune=1000, chains=10, target_accept=0.95, initvals=init)

In [ ]:
with pm.Model() as model5:
    # Priors
    p = pm.Beta('p', 1, 1)

    alpha_1 = pm.Uniform('alpha_1', 0, 1)
    alpha_2 = pm.Uniform('alpha_2', 0, 1)

    p_i = 1 - (1 - p)**t0 * (1 - alpha_1 * p)**t1 * (1 - alpha_1* alpha_2 * p)**t2 

    alpha_12 = pm.Deterministic('alpha_12', alpha_1* alpha_2)
    
    p_i = pm.Deterministic('p_i', p_i)


    # Likelihood
    y_true = pm.Bernoulli('y_obs', p=p_i, observed=y)

    init = {'p': 0.001}

with model5:
    trace5 = pm.sampling.jax.sample_blackjax_nuts(draws=10000, tune=1000, chains=10, target_accept=0.96, initvals=init)

In [ ]:
# samples1 = trace1.posterior.stack(sample=("chain", "draw"))
# samples2 = trace2.posterior.stack(sample=("chain", "draw"))
# samples3 = trace3.posterior.stack(sample=("chain", "draw"))
# samples4 = trace4.posterior.stack(sample=("chain", "draw"))
samples5 = trace5.posterior.stack(sample=("chain", "draw"))

In [ ]:
# p_ind1 = pd.DataFrame(samples1['p_i'].values)
# p_ind2 = pd.DataFrame(samples2['p_i'].values)
# p_ind3 = pd.DataFrame(samples3['p_i'].values)
# p_ind4 = pd.DataFrame(samples4['p_i'].values)
p_ind5 = pd.DataFrame(samples5['p_i'].values)

### Posterior plots

In [ ]:
az.summary(trace5, hdi_prob=.95)

In [ ]:
az.plot_posterior(trace5, round_to=4, hdi_prob=.95);

In [ ]:
az.plot_posterior(trace2, round_to=4, hdi_prob=.95);

In [ ]:
az.plot_posterior(trace3, round_to=4, hdi_prob=.95);

In [ ]:
az.plot_posterior(trace4, round_to=4, hdi_prob=.95);

### ROC curves

In [ ]:
roc_model1 = plot_mean_roc(y, p_ind1)

In [ ]:
roc_model2 = plot_mean_roc(y, p_ind2)

In [ ]:
roc_model3 = plot_mean_roc(y, p_ind3)

In [ ]:
roc_model4 = plot_mean_roc(y, p_ind4)

In [ ]:
roc_model5 = plot_mean_roc(y, p_ind5)

### Confusion matrices

In [ ]:
cm_model1 = calculate_mean_confusion_matrix(y, p_ind1, threshold=0.5)

In [ ]:
cm_model2 = calculate_mean_confusion_matrix(y, p_ind2, threshold=0.5)

In [ ]:
cm_model3 = calculate_mean_confusion_matrix(y, p_ind3, threshold=0.5)

In [ ]:
cm_model4 = calculate_mean_confusion_matrix(y, p_ind4, threshold=0.5)

In [ ]:


cm_model5 = calculate_mean_confusion_matrix(y, p_ind5, threshold=0.5)



In [ ]:
import seaborn as sns
# Define the labels for the confusion matrix
labels = ['Not Infected', 'Infected']

# Plot the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm_model5, annot=True, fmt='.0f', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## Model comparisons

### Model comparison: AIC & BIC 

In [ ]:
results1 = calculate_information_criteria(trace1, model1, y)

print(f"AIC: {results1['AIC']}")
print(f"BIC: {results1['BIC']}")
print(f"WAIC: {results1['WAIC']}")

In [ ]:
# samples1 = trace1.posterior.stack(sample=("chain", "draw"))

# alpha1_samples1 = samples1['alpha_1'].values

# min(alpha1_samples1), max(alpha1_samples1)

In [ ]:
results2 = calculate_information_criteria(trace2, model2, y)

print(f"AIC: {results2['AIC']}")
print(f"BIC: {results2['BIC']}")
print(f"WAIC: {results2['WAIC']}")

In [ ]:
results3 = calculate_information_criteria(trace3, model3, y)

print(f"AIC: {results3['AIC']}")
print(f"BIC: {results3['BIC']}")
print(f"WAIC: {results3['WAIC']}")

In [ ]:
results4 = calculate_information_criteria(trace4, model4, y)

print(f"AIC: {results4['AIC']}")
print(f"BIC: {results4['BIC']}")
print(f"WAIC: {results4['WAIC']}")

In [ ]:
results5 = calculate_information_criteria(trace5, model5, y)

print(f"AIC: {results5['AIC']}")
print(f"BIC: {results5['BIC']}")
print(f"WAIC: {results5['WAIC']}")

### Model comparison: Widely Applicable Information Criterion (WAIC)

In [ ]:
log_likelihood = trace2.log_likelihood
print(log_likelihood)

In [ ]:
log_likelihood_data = trace2['log_likelihood']
print(np.isnan(log_likelihood_data).sum(), "\n\n")
print(np.isinf(log_likelihood_data).sum())

In [ ]:
inf_indices = np.isinf(log_likelihood_data['y_obs']).values
print("Indices des valeurs infinies :", np.where(inf_indices)[0])

In [ ]:
waic1 = pm.waic(trace1, model1)
waic1

In [ ]:
waic2 = pm.waic(trace2, model2)
waic2

In [ ]:
waic3 = pm.waic(trace3, model3)
waic3

In [ ]:
waic4 = pm.waic(trace4, model4)
waic4

### Model comparison: Pareto-smoothed importance sampling leave-one-out cross-validation (PSIS-LOO-CV)

In [ ]:
models = {
    "model1": trace1,
    "model2": trace2,
    "model3": trace3,
    "model4": trace4,
    "model5": trace5
}
model_compare_default = az.compare(models)
model_compare_default

In [ ]:
az.plot_compare(model_compare_default, insample_dev=True);

In [ ]:
model_compare_waic = az.compare(models, ic='waic', method='BB-pseudo-BMA', b_samples=1000)
model_compare_waic

In [ ]:
model_compare_loo = az.compare(models, ic='loo', method='BB-pseudo-BMA', b_samples=1000)
model_compare_loo

### Classification report: F1 score

In [ ]:
f1_model1 = calculate_mean_f1(y, p_ind1, threshold=0.5)

In [ ]:
f1_model2 = calculate_mean_f1(y, p_ind2, threshold=0.5)

In [ ]:
f1_model3 = calculate_mean_f1(y, p_ind3, threshold=0.5)

In [ ]:
f1_model4 = calculate_mean_f1(y, p_ind4, threshold=0.5)

### Saving models' traces

In [ ]:
# with open('trace_model5.pkl', 'wb') as f:
#     pickle.dump(trace5, f)

In [ ]:
# with open('trace_model1.pkl', 'wb') as f:
#     pickle.dump(trace1, f)

# with open('trace_model2.pkl', 'wb') as f:
#     pickle.dump(trace2, f)

# with open('trace_model3.pkl', 'wb') as f:
#     pickle.dump(trace3, f)

# with open('trace_model4.pkl', 'wb') as f:
#     pickle.dump(trace4, f)